<a href="https://colab.research.google.com/github/HimashiRathnayake/STILT/blob/main/STILT_Experiments_Intermediate_Task_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pretrained_model_path = "xlm-roberta-base"
num_labels = 3
intermediate_task = "En-SA" #@param ["Hi-NLI", "En-NLI", "Hi-SA", "En-SA"]
random_state = 42
intermediate_size = 4000
output_dir = "/content/drive/Shareddrives/Research/Transfer Learning/Experiments/"+ str(random_state) +"/" + intermediate_task + "/" + str(intermediate_size)

In [ ]:
print(output_dir)

/content/drive/Shareddrives/Research/Transfer Learning/Experiments/42/En-SA/4000


# Setup

Install Libraries

In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install evaluate
# !pip install sentencepiece

Load Libraries

In [ ]:
import torch
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check GPU

In [ ]:
print("GPU Device name")
torch.cuda.get_device_name(0)

GPU Device name


'Tesla T4'

# Load Hi-SA

In [ ]:
!unzip "/content/drive/Shareddrives/Research/Transfer Learning/Datasets/Hi SA/ReviewSentimentDataset-SentenceClassification.zip"

Archive:  /content/drive/Shareddrives/Research/Transfer Learning/Datasets/Hi SA/ReviewSentimentDataset-SentenceClassification.zip
replace ReviewSentimentDataset-SentenceClassification/ReadMe.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ReviewSentimentDataset-SentenceClassification/ReviewSentimentDataset-SentenceClassification.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
dataset = pd.read_xml("/content/ReviewSentimentDataset-SentenceClassification/ReviewSentimentDataset-SentenceClassification.xml")
dataset = dataset.dropna()
indexRemove = dataset[dataset['polarity'] == 'conflict'].index
dataset.drop(indexRemove , inplace=True)
dataset['polarity'], uniq = pd.factorize(dataset['polarity'])
dataset['polarity'].unique()
dataset = dataset[['polarity', 'text']]
print((dataset['polarity'].value_counts()) / len(dataset) * 100)
np.random.seed(random_state)
dataset = dataset.groupby('polarity', group_keys=False).apply(lambda x: x.sample(frac=5000/len(dataset))).reset_index()
print(len(dataset))
# dataset = dataset.groupby('polarity', group_keys=False).apply(lambda x: x.sample(frac=0.5001)).reset_index()
# dataset = dataset.groupby('polarity', group_keys=False).apply(lambda x: x.sample(frac=0.5))
# print((dataset['polarity'].value_counts()) / len(dataset) * 100)
# print(len(dataset))
X = dataset['text'].values.tolist()
y = dataset['polarity'].values.tolist()

1    43.802601
0    42.578424
2    13.618975
Name: polarity, dtype: float64
5000


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state = random_state)

In [ ]:
len(X_train), len(X_valid), len(X_test)

(4000, 500, 500)

In [ ]:
hi_sa_train_df = pd.DataFrame({"text": X_train, "label": y_train})
hi_sa_valid_df = pd.DataFrame({"text": X_valid, "label": y_valid})

In [ ]:
hi_sa_train_dataset = Dataset.from_pandas(hi_sa_train_df)
hi_sa_valid_dataset = Dataset.from_pandas(hi_sa_valid_df)

In [ ]:
dataset = DatasetDict()
dataset["train"] = hi_sa_train_dataset
dataset["validation"] = hi_sa_valid_dataset
# dataset["test"] = test_data
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 500
    })
})

# Huggingface Datasets Setup

Hi/En NLI

In [ ]:
if (intermediate_task != "Hi-SA"):
  if (intermediate_task == "Hi-NLI"):
    dataset = load_dataset("Divyanshu/indicxnli", "hi")
  elif (intermediate_task == "En-NLI"):
    dataset = load_dataset("glue", "mnli")
  elif (intermediate_task == "En-SA"):
    dataset = load_dataset("SetFit/sst5")

  np.random.seed(random_state)
  train_df = dataset["train"].to_pandas()
  valid_df = dataset["validation"].to_pandas() #validation_matched

  if (intermediate_task == "En-SA"):
    train_df['label'] = train_df['label'].replace([1], 0)
    train_df['label'] = train_df['label'].replace([2], 1)
    train_df['label'] = train_df['label'].replace([3], 2)
    train_df['label'] = train_df['label'].replace([4], 2)
    valid_df['label'] = valid_df['label'].replace([1], 0)
    valid_df['label'] = valid_df['label'].replace([2], 1)
    valid_df['label'] = valid_df['label'].replace([3], 2)
    valid_df['label'] = valid_df['label'].replace([4], 2)
  # En SA - 0.4682, 0.454
  # Hi NLI - 0.0127334, 0.201
  # En NLI - 0.0127315, 0.051
  train_data = Dataset.from_pandas(train_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=4000/len(train_df))).reset_index())
  # train_data = Dataset.from_pandas(train_data.to_pandas().groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.5)).reset_index())
  # train_data = Dataset.from_pandas(train_data.to_pandas().groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.4)))
  valid_data = Dataset.from_pandas(valid_df.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=500/len(valid_df))).reset_index())
  # valid_data = Dataset.from_pandas(valid_data.to_pandas().groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.5)).reset_index())  
  # valid_data = Dataset.from_pandas(valid_data.to_pandas().groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.5)))  
  # test_data = Dataset.from_pandas(dataset["test_matched"].to_pandas().groupby('label', group_keys=False).apply(lambda x: x.sample(frac=0.051)).reset_index())  
  # print(train_data.unique("label"), valid_data.unique("label"))
  print(len(train_data), len(valid_data))
  # print(train_data['hypothesis'][:10])
  # print(valid_data['hypothesis'][:10])
  dataset["train"] = train_data
  dataset["validation"] = valid_data
  dataset

  0%|          | 0/3 [00:00<?, ?it/s]

4000 500


# Tokenization

In [ ]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_function(examples):
    # return tokenizer(examples["premise"], examples['hypothesis'], padding="max_length", truncation=True)
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train_dataset = tokenized_datasets["train"].shuffle(seed=random_state)
eval_dataset = tokenized_datasets["validation"].shuffle(seed=random_state)

# Training

In [ ]:
def model_init():
  return XLMRobertaForSequenceClassification.from_pretrained(pretrained_model_path, num_labels=num_labels, ignore_mismatched_sizes=True)

In [ ]:
def compute_metrics(eval_pred):
    accuracy = load_metric("accuracy")
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    precision = precision.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = recall.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = f1.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    return {"accuracy":accuracy, "precision": precision, "recall": recall, "f1": f1}    

In [ ]:
training_args = TrainingArguments(
    num_train_epochs = 3,
    learning_rate = 2e-5, # 1e-5
    output_dir=output_dir, 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_f1",
    load_best_model_at_end=True,
    seed = random_state,
    data_seed = random_state
)

In [ ]:
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d161e8f5f6f2ed433fb4023d6cb34146506b3f/config.json
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loadi

In [ ]:
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d161e8f5f6f2ed433fb4023d6cb34146506b3f/config.json
Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loadi

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.060900,1.062673,0.388000,0.150544,0.388000,0.216922
2,1.054300,1.061185,0.404000,0.163216,0.404000,0.232501
3,1.052000,1.061978,0.404000,0.163216,0.404000,0.232501


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: label_text, index, text. If label_text, index, text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /content/drive/Shareddrives/Rese

TrainOutput(global_step=1500, training_loss=1.0557190755208334, metrics={'train_runtime': 1342.4929, 'train_samples_per_second': 8.939, 'train_steps_per_second': 1.117, 'total_flos': 3157361012736000.0, 'train_loss': 1.0557190755208334, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: label_text, index, text. If label_text, index, text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 1.061185359954834,
 'eval_accuracy': 0.404,
 'eval_precision': 0.163216,
 'eval_recall': 0.404,
 'eval_f1': 0.2325014245014245,
 'eval_runtime': 17.6735,
 'eval_samples_per_second': 28.291,
 'eval_steps_per_second': 3.565,
 'epoch': 3.0}

# Save Model

In [ ]:
trainer.save_model(output_dir+"/model")

Saving model checkpoint to /content/drive/Shareddrives/Research/Transfer Learning/Experiments/42/En-SA/4000/model
Configuration saved in /content/drive/Shareddrives/Research/Transfer Learning/Experiments/42/En-SA/4000/model/config.json
Model weights saved in /content/drive/Shareddrives/Research/Transfer Learning/Experiments/42/En-SA/4000/model/pytorch_model.bin
